#### 데이터를 CSV파일로 저장 <hr>

In [4]:
import csv

csvFile = open('test.csv', 'w', encoding = 'UTF-8')
try:
    writer = csv.writer(csvFile)
    writer.writerow(('number', 'number+2', '(number+2)^2'))
    
    for i in range(10):
        writer.writerow((i, i+2, pow(i+2, 2)))
except Exception as e:
    print(e)
finally:
    csvFile.close()

In [3]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
# 두 개의 테이블 중에 첫 번째 테이블 사용
table = bs.find_all('table', {'class' : 'wikitable'})[0]
rows = table.find_all('tr')

csvFile = open('editors.csv', 'wt', encoding = 'utf-8')  # t : text_mode
writer = csv.writer(csvFile)

try:
    for row in rows:
        csvRow = []
        for cell in row.find_all(['th', 'td']):
            print(cell.text.strip())
            csvRow.append(cell.text.strip())
        writer.writerow(csvRow)
finally:
    csvFile.close()
                       

Name
Developer
Initial release
Latest release
Programming language
Cost (US$)
License
GUI
TUI or CLI
Version
Date
Acme
Rob Pike
1993
Plan 9 and Inferno

C
No cost
MITGPL-2.0-onlyLPL-1.02


AkelPad
Alexey KuznetsovAlexander Shengalts
2003


C
No cost
BSD-2-Clause


Alphatk
Vince Darley
1999
8.3.3[1]
2004-12-10

$40
Proprietary, with BSD components


Atom
GitHub
2014
1.63.1[2]
2022-11-23
HTML, CSS, JavaScript, C++
No cost
MIT


BBEdit
Rich Siegel
1992
15.0[3]
2024-01-09
Objective-C, Objective-C++
No cost for most features, $49.99 for full version
Proprietary


Bluefish
Bluefish Development Team
1999
2.2.14[4]
2023-06-03
C
No cost
GPL-3.0-or-later


Brackets
Adobe Systems
2012
2.2.1[5]
2023-03-22
HTML, CSS, JavaScript, C++
No cost
MIT


Coda
Panic
2007
2.7.7[6]
2020-11-05
Objective-C
$99
Proprietary


ConTEXT
ConTEXT Project Ltd
1999


Object Pascal (Delphi)
No cost
BSD-3-Clause


Crimson Editor
Ingyu Kang
1999
3.72-r286m[7]
2011-10-01
C++
No cost
Proprietary


CudaText
UVViewSoft[a]
2015

- 테이블 데이터를 CSV로 저장 : html_table_parser 사용 예제 #1

In [8]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parse

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')

# 두 개의 테이블 중에 첫 번째 테이블 사용 : find_all() 사용
# table = bs.find_all('talbe', {'class' : 'wikitable'})[0]
table = bs.find('table', {'class' : 'wikitable'})
table_data = parse.make2d(table)  # 2차원 리스트 형태로 변환

# 테이블의 2행을 출력
print('[0]:', table_data[0])
print('[1]:', table_data[1])

[0]: ['Name', 'Developer', 'Initial release', 'Latest release', 'Latest release', 'Programming language', 'Cost (US$)', 'License', 'GUI', 'TUI or CLI']
[1]: ['Name', 'Developer', 'Initial release', 'Version', 'Date', 'Programming language', 'Cost (US$)', 'License', 'GUI', 'TUI or CLI']


- 테이블 데이터를 CSV로 저장 : html_table_parser 사용 예제 #2

In [11]:
import pandas as pd

# Pandas DataFrame으로 저장 (2행부터 데이터 저장, 1행은 column 이름으로 사용)
df = pd.DataFrame(table_data[2:], columns = table_data[1])
print(df.head())

# csv 파일로 저장
csvFile = open('editors1.csv', 'w', encoding = 'utf-8')  # t : text mode
writer = csv.writer(csvFile)

for row in table_data:
    writer.writerow(row)

csvFile.close()

      Name                            Developer Initial release  \
0     Acme                             Rob Pike            1993   
1  AkelPad  Alexey KuznetsovAlexander Shengalts            2003   
2  Alphatk                         Vince Darley            1999   
3     Atom                               GitHub            2014   
4   BBEdit                          Rich Siegel            1992   

              Version        Date        Programming language  \
0  Plan 9 and Inferno                                       C   
1                                                           C   
2            8.3.3[1]  2004-12-10                               
3           1.63.1[2]  2022-11-23  HTML, CSS, JavaScript, C++   
4             15.0[3]  2024-01-09  Objective-C, Objective-C++   

                                          Cost (US$)  \
0                                            No cost   
1                                            No cost   
2                                     

- 파이썬과 통합 : 예제 #1
    - page 테이블에서 id값이 2인 데이터 가져오기

In [16]:
import pymysql

conn = pymysql.connect(host = 'localhost', user = 'root', password = '2357', db = 'scraping', charset = 'utf8')

cur = conn.cursor()
cur.execute('use scraping')
cur.execute('select * from pages where id = 2')

print(cur.fetchone())
cur.close()
conn.close()

(2, 'second page title', 'This is the second test page content', datetime.datetime(2024, 2, 13, 10, 53, 26))


- 파이썬과 통합 : 예제 #2
    - 위키피디아의 자료를 MySQL에 저장
        - Kevin Bacon에 연결된 모든 링크 가져오기

In [18]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import pymysql
import re

conn = pymysql.connect(host = 'localhost', user = 'root', password = '2357', db = 'scraping', charset = 'utf8')

cur = conn.cursor()
random.seed(None)

def store(title, content):
    cur.execute('insert into pages(title, content) values ("%s", "%s")', (title, content))
    cur.connection.commit()


In [21]:
def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org' + articleUrl)
    bs = BeautifulSoup(html, 'html.parser')
    title =	bs.find('h1').text
    content	= bs.find('div', {'id':'mw-content-text'}).find('p').text
	# find()로 검색된 데이터를 데이터베이스에 저장
    store(title, content)
    return bs.find('div', {'id':'bodyContent'}).\
		find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))
links =	getLinks('/wiki/Kevin_Bacon')
try:
	while len(links)	>	0:
		newArticle = links[random.randint(0, len(links)-1)].attrs['href']
		print(newArticle)
		links =	getLinks(newArticle)
finally:
	cur.close()
	conn.close()

/wiki/Albert_Finney
/wiki/Tom_Jones_(1963_film)
/wiki/Five_Easy_Pieces
/wiki/Academy_Awards
/wiki/Art_Deco
/wiki/Ultra-Lettrist
/wiki/Situationist_International
/wiki/Liberal_International
/wiki/Reform_Movement_(Guatemala)
/wiki/Democratic_Convergence_of_Catalonia
/wiki/Together_for_Catalonia_(2020)
/wiki/Sumar_(electoral_platform)
/wiki/Ada_Colau
/wiki/Barcelona_City_Council
/wiki/1911_Barcelona_City_Council_election
/wiki/1991_Barcelona_City_Council_election
/wiki/1991_Spanish_regional_elections
/wiki/1987_European_Parliament_election_in_Spain
/wiki/2004_European_Parliament_election_in_Ireland
/wiki/2024_European_Parliament_election_in_Ireland
/wiki/2014_European_Parliament_election_in_Bulgaria
/wiki/1996_European_Parliament_election_in_Finland
/wiki/2019_European_Parliament_election_in_Austria
/wiki/2024_European_Parliament_election_in_Luxembourg
/wiki/Open_list#Most_open
/wiki/Elections_in_the_Czech_Republic
/wiki/Jaroslav_Ba%C5%A1ta
/wiki/KDU-%C4%8CSL
/wiki/Pavel_Fischer
/wiki/Kar

Traceback (most recent call last):
  File "c:\Users\xnnly\.conda\envs\TEST\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\xnnly\AppData\Local\Temp\ipykernel_9668\3034308677.py", line 15, in <module>
    links =	getLinks(newArticle)
  File "C:\Users\xnnly\AppData\Local\Temp\ipykernel_9668\3034308677.py", line 5, in getLinks
    content	= bs.find('div', {'id':'mw-content-text'}).find('p').text
  File "c:\Users\xnnly\.conda\envs\TEST\lib\site-packages\bs4\element.py", line 2001, in find
  File "c:\Users\xnnly\.conda\envs\TEST\lib\site-packages\bs4\element.py", line 2030, in find_all
  File "c:\Users\xnnly\.conda\envs\TEST\lib\site-packages\bs4\element.py", line 837, in _find_all
  File "c:\Users\xnnly\.conda\envs\TEST\lib\site-packages\bs4\element.py", line 2070, in descendants
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent cal